# Capstone Project

## Import relevant libraries

In [1]:
import os
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

## Read data 

In [2]:
data_dir = "engineered-data"
wege_df = pd.read_csv(os.path.join(data_dir, "wege.csv"))
bova_df = pd.read_csv(os.path.join(data_dir, "bova.csv"))
dfs = [wege_df, bova_df]
for df in dfs:
    df.index = df["Date"]


### Columns in dataset

In [3]:
print(wege_df.columns.values)

['Date' 'Open' 'Max' 'Min' 'Close' 'Volume [BRL]' 'Volume [shares]'
 'IFR (RSI) [9]' 'True Range [1]' 'Moving Average A [21]'
 'Moving Average E [9]' 'IsLocalPeak' 'IsLocalTrough' 'derivative_peaks'
 'derivative_troughs' 'bear_harami' 'bull_harami' 'bear_engulfing'
 'bull_engulfing' 'hammer' 'shooting_star' 'D+1']


## Clean Data

Since we will use a single dataset (IBOVESPA index will be used to predict WEGE price), we will rename BOVA11's columns

In [4]:
 # Some features are not going to be used by the model. For instance, we cannot know whether a day was a local peak/trough untill its following day,
 # so we cannot use it as an input for predicting the next day.
cols_to_drop = ["IsLocalPeak","IsLocalTrough"]
for df in dfs:
    for col in cols_to_drop:
        df.drop(col, axis="columns", inplace=True, errors="ignore")
bova_df.drop("D+1", axis="columns", inplace=True)
dict_colnames = {}
for column in bova_df.columns:
    dict_colnames[column] = "bova_" + column
bova_df.rename(columns=dict_colnames, inplace=True)


In [5]:
complete_df = pd.concat([bova_df, wege_df], axis=1, sort=False)
complete_df.index = complete_df["Date"]
complete_df.dropna(inplace=True)
complete_df.drop("bova_Date", axis="columns", inplace=True)
complete_df.drop("16/10/2020",inplace=True, errors="ignore") # since we don't have D+1 for our last day in the dataset
print(complete_df.columns.values)
complete_df.tail()

['bova_Open' 'bova_Max' 'bova_Min' 'bova_Close' 'bova_Volume [BRL]'
 'bova_Volume [shares]' 'bova_IFR (RSI) [9]' 'bova_True Range [1]'
 'bova_Moving Average A [21]' 'bova_Moving Average E [9]'
 'bova_derivative_peaks' 'bova_derivative_troughs' 'bova_bear_harami'
 'bova_bull_harami' 'bova_bear_engulfing' 'bova_bull_engulfing'
 'bova_hammer' 'bova_shooting_star' 'Date' 'Open' 'Max' 'Min' 'Close'
 'Volume [BRL]' 'Volume [shares]' 'IFR (RSI) [9]' 'True Range [1]'
 'Moving Average A [21]' 'Moving Average E [9]' 'derivative_peaks'
 'derivative_troughs' 'bear_harami' 'bull_harami' 'bear_engulfing'
 'bull_engulfing' 'hammer' 'shooting_star' 'D+1']


,bova_Open,bova_Max,bova_Min,bova_Close,bova_Volume [BRL],bova_Volume [shares],bova_IFR (RSI) [9],bova_True Range [1],bova_Moving Average A [21],bova_Moving Average E [9],...,Moving Average E [9],derivative_peaks,derivative_troughs,bear_harami,bull_harami,bear_engulfing,bull_engulfing,hammer,shooting_star,D+1
Date,,,,,,,,,,,,,,,,,,,,,
08/10/2020,92.18,94.32,92.01,94.27,1.098641e+09,11741375.0,56.25,2.37,93.358571,92.476244,...,69.782360,0.103333,0.0400,0.0,0.0,0.0,0.0,0.0,0.0,76.90
09/10/2020,94.13,94.99,93.50,93.65,8.611634e+08,9149800.0,52.96,1.49,93.292381,92.710995,...,71.205888,0.103333,0.0400,0.0,0.0,0.0,0.0,0.0,0.0,79.15
13/10/2020,94.10,95.35,93.65,94.80,9.415492e+08,9960394.0,58.07,1.70,93.307619,93.128796,...,72.794711,0.103333,0.0400,0.0,0.0,0.0,0.0,0.0,0.0,80.58
14/10/2020,95.01,95.86,95.01,95.59,1.092614e+09,11431930.0,61.32,1.06,93.263810,93.621037,...,74.351768,0.103333,0.0400,0.0,0.0,0.0,0.0,0.0,0.0,80.77
15/10/2020,94.36,95.80,94.05,95.34,9.081088e+08,9544784.0,59.67,1.75,93.210476,93.964830,...,75.635415,0.103333,1.2925,0.0,0.0,0.0,0.0,0.0,0.0,82.05


In [6]:
print(wege_df.index)
print(bova_df.index)
print(complete_df.index)

Index(['04/07/1994', '05/07/1994', '06/07/1994', '07/07/1994', '08/07/1994',
       '11/07/1994', '12/07/1994', '27/07/1994', '28/07/1994', '03/08/1994',
       ...
       '02/10/2020', '05/10/2020', '06/10/2020', '07/10/2020', '08/10/2020',
       '09/10/2020', '13/10/2020', '14/10/2020', '15/10/2020', '16/10/2020'],
      dtype='object', name='Date', length=5660)
Index(['02/12/2008', '03/12/2008', '04/12/2008', '05/12/2008', '08/12/2008',
       '09/12/2008', '10/12/2008', '11/12/2008', '12/12/2008', '15/12/2008',
       ...
       '02/10/2020', '05/10/2020', '06/10/2020', '07/10/2020', '08/10/2020',
       '09/10/2020', '13/10/2020', '14/10/2020', '15/10/2020', '16/10/2020'],
      dtype='object', name='Date', length=2936)
Index(['12/12/2008', '15/12/2008', '16/12/2008', '17/12/2008', '18/12/2008',
       '19/12/2008', '22/12/2008', '23/12/2008', '26/12/2008', '29/12/2008',
       ...
       '01/10/2020', '02/10/2020', '05/10/2020', '06/10/2020', '07/10/2020',
       '08/10/2020', '

## Scale data

In [14]:
def scale_data(df, col_name, feature_range):
    scaler = MinMaxScaler(feature_range=feature_range)
    scaled_values = scaler.fit_transform(df[col_name].values.reshape(-1, 1))
    return (scaler, scaled_values)

In [15]:
dict_scalers = {}

for column_name in complete_df.columns:
    if pd.api.types.is_numeric_dtype(complete_df[column_name]):
        feature_range = (-1, 1) if min(complete_df[column_name].values) < 0 else (0, 1)
        dict_scalers[column_name] = scale_data(complete_df, column_name, feature_range)
print(dict_scalers)

{'bova_Open': (MinMaxScaler(copy=True, feature_range=(0, 1)), array([[0.01305615],
       [0.03790087],
       [0.03866143],
       ...,
       [0.73241222],
       [0.74394727],
       [0.73570795]])), 'bova_Max': (MinMaxScaler(copy=True, feature_range=(0, 1)), array([[0.03394787],
       [0.03966942],
       [0.04310235],
       ...,
       [0.74685315],
       [0.75333757],
       [0.7525747 ]])), 'bova_Min': (MinMaxScaler(copy=True, feature_range=(0, 1)), array([[0.01628623],
       [0.03385483],
       [0.04526802],
       ...,
       [0.74275455],
       [0.76019492],
       [0.74788407]])), 'bova_Close': (MinMaxScaler(copy=True, feature_range=(0, 1)), array([[0.03377349],
       [0.02691722],
       [0.04507364],
       ...,
       [0.7408583 ],
       [0.75088878],
       [0.74771458]])), 'bova_Volume [BRL]': (MinMaxScaler(copy=True, feature_range=(0, 1)), array([[0.000688  ],
       [0.00089234],
       [0.00705361],
       ...,
       [0.28562846],
       [0.33146431],
      

In [28]:

#from: machinelearningmastery.com/blog/

def time_series_to_supervised(data, n_lag=1, n_fut=1, selLag=None, selFut=None, dropnan=True):
    """
    Converts a time series to a supervised learning data set by adding time-shifted prior and future period
    data as input or output (i.e., target result) columns for each period
    :param data:  a series of periodic attributes as a list or NumPy array
    :param n_lag: number of PRIOR periods to lag as input (X); generates: Xa(t-1), Xa(t-2); min= 0 --> nothing lagged
    :param n_fut: number of FUTURE periods to add as target output (y); generates Yout(t+1); min= 0 --> no future periods
    :param selLag:  only copy these specific PRIOR period attributes; default= None; EX: ['Xa', 'Xb' ]
    :param selFut:  only copy these specific FUTURE period attributes; default= None; EX: ['rslt', 'xx']
    :param dropnan: True= drop rows with NaN values; default= True
    :return: a Pandas DataFrame of time series data organized for supervised learning
    NOTES:
    (1) The current period's data is always included in the output.
    (2) A suffix is added to the original column names to indicate a relative time reference: e.g., (t) is the current
        period; (t-2) is from two periods in the past; (t+1) is from the next period
    (3) This is an extension of Jason Brownlee's series_to_supervised() function, customized for MFI use
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    origNames = df.columns
    cols, names = list(), list()
    # include all current period attributes
    cols.append(df.shift(0))
    names += [('%s' % origNames[j]) for j in range(n_vars)]
 
    # lag any past period attributes (t-n_lag,...,t-1)
    n_lag = max(0, n_lag)  # force valid number of lag periods
    for i in range(n_lag, 0, -1):
        suffix= '(t-%d)' % i
        if (None == selLag):   # copy all attributes from PRIOR periods?
            cols.append(df.shift(i))
            names += [('%s%s' % (origNames[j], suffix)) for j in range(n_vars)]
        else:
            for var in (selLag):
                cols.append(df[var].shift(i))
                names+= [('%s%s' % (var, suffix))]
 
#     # include future period attributes (t+1,...,t+n_fut)
#     n_fut = max(n_fut, 0)  # force valid number of future periods to shift back
#     for i in range(1, n_fut + 1):
#         suffix= '(t+%d)' % i
#         if (None == selFut):  # copy all attributes from future periods?
#             cols.append(df.shift(-i))
#             names += [('%s%s' % (origNames[j], suffix)) for j in range(n_vars)]
#         else:  # copy only selected future attributes
#             for var in (selFut):
#                 cols.append(df[var].shift(-i))
#                 names += [('%s%s' % (var, suffix))]
    # combine everything
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values introduced by lagging
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [29]:
train_df = time_series_to_supervised(complete_df, n_lag=10, n_fut=0, selLag=None, selFut=None, dropnan=True)

In [30]:
for col in train_df.columns.values:
   print(col) 

bova_Open
bova_Max
bova_Min
bova_Close
bova_Volume [BRL]
bova_Volume [shares]
bova_IFR (RSI) [9]
bova_True Range [1]
bova_Moving Average A [21]
bova_Moving Average E [9]
bova_derivative_peaks
bova_derivative_troughs
bova_bear_harami
bova_bull_harami
bova_bear_engulfing
bova_bull_engulfing
bova_hammer
bova_shooting_star
Date
Open
Max
Min
Close
Volume [BRL]
Volume [shares]
IFR (RSI) [9]
True Range [1]
Moving Average A [21]
Moving Average E [9]
derivative_peaks
derivative_troughs
bear_harami
bull_harami
bear_engulfing
bull_engulfing
hammer
shooting_star
D+1
bova_Open(t-10)
bova_Max(t-10)
bova_Min(t-10)
bova_Close(t-10)
bova_Volume [BRL](t-10)
bova_Volume [shares](t-10)
bova_IFR (RSI) [9](t-10)
bova_True Range [1](t-10)
bova_Moving Average A [21](t-10)
bova_Moving Average E [9](t-10)
bova_derivative_peaks(t-10)
bova_derivative_troughs(t-10)
bova_bear_harami(t-10)
bova_bull_harami(t-10)
bova_bear_engulfing(t-10)
bova_bull_engulfing(t-10)
bova_hammer(t-10)
bova_shooting_star(t-10)
Date(t-10)

### Split train/test data

We will use all the available data up untill October 15th 2019 as our training set. <br>
The remaining days, which correspond to one year of operations will be our test set.